In [5]:
# !pip install websockets

In [6]:
import pymssql
#from vosk import Model, KaldiRecognizer, SetLogLevel
import wave
import json
import asyncio
import websockets

In [7]:
# cpu approach
model_path = '/mnt/share/audio_call/model_v0/model'
model = Model(model_path)

NameError: name 'Model' is not defined

In [8]:
def connect_sql():

    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'
    
    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()
    
    return pymssql.connect(
        server=sql_server,
        user=sql_login,
        password=sql_pass,
        database=sql_name,
    )

In [19]:
def get_file(source_id):
    
    conn = connect_sql()
    cursor = conn.cursor()
    
    # middle date
    sql_query = "select"
    sql_query += " CAST(AVG(CAST(record_date-0.5 AS FLOAT)) AS DATETIME),"
    sql_query += " CAST(AVG(CAST(record_date+0.1 AS FLOAT)) AS DATETIME)"
    sql_query += " from queue"
    sql_query += " where source_id = '"+source_id+"' and duration>600;"
    cursor.execute(sql_query)
    for row in cursor.fetchall():
        min_time = str(row[0])[:19]
        max_time = str(row[1])[:19]
        print(min_time)
    
    # file name and path from queue
    sql_query = "select top 1 filepath, filename from queue "
    sql_query += "where source_id = '"+source_id+"' "
    sql_query += " and record_date > '" + min_time + "' "
    sql_query += " and record_date < '" + max_time + "' "
    sql_query += " order by record_date, filename;"
    cursor.execute(sql_query)
    for row in cursor.fetchall():
        original_file_path = row[0]
        original_file_name = row[1]
        
    return original_file_path, original_file_name

In [6]:
#original_file_path, original_file_name = get_file('1')
#original_file_path, original_file_name

### MRM records

In [20]:
original_file_path, original_file_name = get_file('2')

2021-08-22 09:21:44


In [21]:
original_file_path, original_file_name

('/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/',
 'a2021-08-22t09:21:48b_c9685337878d_e9639291714f_g1629613306.101424h-in.wav')

In [57]:
def cpu_approach(model, original_file_path, original_file_name):
    # read file
    wf = wave.open(original_file_path + original_file_name, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    phrases_count = 0
    confidences = []
    phrases = []
    while True:

        conf_score = []

        data = wf.readframes(4000)
        if len(data) == 0:
            break

        if rec.AcceptWaveform(data):
            accept = json.loads(rec.Result())
            print('*', accept)
            if accept['text'] != '':

                accept_start = str(accept['result'][0]['start'])
                accept_end = accept['result'][-1:][0]['end']
                accept_text = str(accept['text'])

                for result_rec in accept['result']:
                    conf_score.append(float(result_rec['conf']))
                conf_mid = str(sum(conf_score)/len(conf_score))
                confidences.append(sum(conf_score)/len(conf_score))
                print(accept_start, accept_text)
                phrases.append({
                    'time': accept_start,
                    'conf': conf_mid,
                    'text': accept_text
                })
                phrases_count += 1
    print('end')
    return phrases

In [32]:
phrases_cpu = cpu_approach(model, original_file_path, original_file_name)
phrases_cpu

* {'text': ''}
end


[]

In [30]:
#async def gpu_approach(original_file_path, original_file_name):
# server_ip = '10.2.5.212'
# uri = 'ws://' + server_ip + ':2700'
original_file_name = 'in_9267682610_2021-08-23-11-49-17rxtx_R.wav'
server_ip = '192.168.1.23'
uri = 'ws://' + server_ip + ':2700'
original_file_path = ''
phrases_count = 0
confidences = []
phrases = []
async with websockets.connect(uri) as websocket:
    wf = open(original_file_path + original_file_name, "rb")
    while True:
        conf_score = []
        data = wf.read(16000)
        if len(data) == 0:
            break
        await websocket.send(data)
        accept = json.loads(await websocket.recv())
        print('*', accept)
        if len(accept)>1 and accept['text'] != '':

            accept_start = str(accept['result'][0]['start'])
            accept_end = accept['result'][-1:][0]['end']
            accept_text = str(accept['text'])

            for result_rec in accept['result']:
                conf_score.append(float(result_rec['conf']))
            conf_mid = str(sum(conf_score)/len(conf_score))
            confidences.append(sum(conf_score)/len(conf_score))
            print(accept_start, accept_text)
            phrases.append({
                'time': accept_start,
                'conf': conf_mid,
                'text': accept_text
            })
            #phrases_count += 1
        """if len(accept)>1 and accept['text'] != '':
            pass
            #file_data.append(accept['text'])
            #textfile.write(str(accept['text'])+'\n')
        else:
            [[key, value]] = accept.items()
            if key == 'text' and len(value):
                print('===', value)"""
        
    await websocket.send('{"eof" : 1}')
    print(await websocket.recv())
            
#        return accept
#asyncio.get_event_loop().run_until_complete(
#    gpu_approach(original_file_path, original_file_name)
#)

* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': 'как'}
* {'result': [{'conf': 0.988311, 'end': 11.22, 'start': 10.89, 'word': 'как'}], 'text': 'как'}
10.89 как
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'result': [{'conf': 0.525621, 'end': 13.44, 'start': 13.02, 'word': 'а'}], 'text': 'а'}
13.02 а
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': 'ага'}
* {'result': [{'conf': 1.0, 'end': 22.56, 'start': 22.08, 'word': 'ага'}], 'text': 'ага'}
22.08 ага
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': 'на'}
* {'result': [{'conf': 0.878577, 'end': 32.85, 'start': 32.52, 'word': 'на'}], 'text': 'на'}
32.52 на
* {'partial': ''}
* {'partial': ''}
*

* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'result': [{'conf': 0.648383, 'end': 288.96, 'start': 288.33, 'word': 'хорошо'}], 'text': 'хорошо'}
288.33 хорошо
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'result': [{'conf': 0.6924, 'end': 315.51, 'start': 315.119795, 'word': 'про'}], 'text': 'п

In [28]:
phrases

[{'time': '17.64', 'conf': '0.461058', 'text': 'рога'},
 {'time': '24.72', 'conf': '0.568129', 'text': 'о'},
 {'time': '46.05', 'conf': '0.776835', 'text': 'да'},
 {'time': '61.98', 'conf': '0.50251', 'text': 'берег'},
 {'time': '88.62', 'conf': '0.712986', 'text': 'вафли'},
 {'time': '102.45', 'conf': '1.0', 'text': 'гнев'},
 {'time': '106.05', 'conf': '0.295827', 'text': 'родился'},
 {'time': '109.05', 'conf': '0.789847', 'text': 'нет'},
 {'time': '116.19', 'conf': '1.0', 'text': 'два'},
 {'time': '118.53', 'conf': '0.529292', 'text': 'да'},
 {'time': '126.6', 'conf': '0.188348', 'text': 'билеты'},
 {'time': '132.0', 'conf': '0.396566', 'text': 'уютно'},
 {'time': '136.83', 'conf': '0.301844', 'text': 'ты'},
 {'time': '140.34', 'conf': '0.623102', 'text': 'син'},
 {'time': '153.06', 'conf': '0.8252575', 'text': 'свет и'},
 {'time': '169.14', 'conf': '1.0', 'text': 'его'},
 {'time': '172.02', 'conf': '0.6185196', 'text': 'миров и день и города'},
 {'time': '179.697041', 'conf': '0.328